![snap](https://media.giphy.com/media/l0NgQgSHKlhCyQYDu/giphy.gif)

# Docker Volumes 

## What you will learn in this course 🧐🧐

Volumes are a way to decouple data from your container. It has a lot of interesting usecases. For example, you might want to seperate source files from your container so that you can reuse them later on after you have stopped your container. Another example is to be able to write code on your local machine that you can modify without having to rebuild your image or rerun your container. In this course, we will learn:

* What is a volume 
* How to set up a volume 

## What is a volume 

### Example of volumes usecase

To understand the purpose of using volumes, let's take an example. Imagine that you have a folder called `app` that is organized like this:

```
├── Dockerfile
└── app.py
```

Within your `app.py` is a very simple script:

```python 
# Within your app.py

print("Hello world from Jedha!")
```

And within your `Dockerfile`: 

```Dockerfile
# Within your Dockerfile 
FROM python:3.9.9-alpine

WORKDIR /home/app

COPY app.py app.py
```

Now build your image: 

`docker build . -t jedha/volumes_demo`

and run your container using this command:

`docker run -it jedha/volumes_demo sh`

Within your container, try to run `app.py` by running `python app.py` you should see the following output: `Hello world from Jedha!`. 

Now, opened VSCode and try to modify the code in `app.py` and rerun `python app.py` you should still see: `Hello world from Jedha!` 🤔. 

With this code, this only way you could reflect your changes would be to stop your container, build a new image and then rerun your container. Which is quite tedious... 

So how could we solve that problem? 🥁 ...

With **volumes**!

### Volumes definition 

A volume is a way to persist data outside of a container. This works perfectly with code for example. 

![snap](https://full-stack-assets.s3.eu-west-3.amazonaws.com/Deployment/Volumes.svg)

In the above diagram for example, you can have one volume applied to several containers. Eventhough a container is stopped or removed, the **volume will persist** which is what is great about it! 

Now, there are several types of volumes: 

* `bind mounts`
* `volumes` 
* `tmpfs mounts`
* `named pipes`

With `bind mounts` you will use it to be able to modify code in a running container. You will need to specify the absolute path of where your data is living within your host machine. That is why it is perfect for this usecase

On the other hand, `volumes` are more of general use. You will create it using `docker volumes create VOLUME_NAME` and then attach it to your container. This is perfect when you want to have persisting data like `.csv` files. 

The other two are definitely less used so we won't develop them in this course. Feel free to read more here though: https://docs.docker.com/storage/

## Bind Mounts 🏇

Let's start with the first example: `bind mounts`. Let's take back our example within our `app` directory:

```
├── Dockerfile
└── app.py
```

Now run: 

```shell
docker run -it -v "$(pwd):/home/app" jedha/volumes_demo sh
```

> for windows classic shell 
>```shell
>docker run -it -v "%cd%:/home/app" volumes sh
>```

> for windows powershell
>```shell
>docker run -it -v "$(pwd):/home/app" volumes sh
>```

Type then `ls` and you should see:

```
Dockerfile  app.py
```

Run `python app.py` and you should see:

`Hello World from Jedha!`

Open VSCode, add this code within your `app.py` file:

```python 
# Within app.py 
print("Hello World from Jedha!")
print("This is Volumes magical powers 🪄")
```

Now rerun `app.py` without stopping your container and you should see now the difference:

```
Hello World from Jedha!
This is Volumes magical powers 🪄
```

Cool right? And really simple!

## Volumes 🔉

As stated before, volumes are more generic, instead of living on your host machine, you will need to create them using this command: 

`docker volume create VOLUME_NAME` 

This volume will then live in a folder managed by Docker (usually at `/var/lib/docker/volumes/`). You will then be able to mount it onto any container like this: 

```
docker run \ 
--mount source=VOLUME_NAME,target=CONTAINER_FOLDER_TO_PERSIST_DATA \
IMAGE_NAME
```

Let's take an example. Back to our `app` directory: 

```
├── Dockerfile
└── app.py
```

Let's create a volume that we will call `my_first_volume`

`docker volume create my_first_volume`

Then run: 

```
docker run -it \ 
--mount source=my_first_volume,target=/home/app \
jedha/volumes_demo \
sh
```

Now within your container, create a new file:

`touch hello.py`

Stop your container and rerun:

```
docker run -it \ 
--mount source=my_first_volume,target=/home/app \
jedha/volumes_demo \
sh
```

> for windows' command promt
> ```
> docker run -it \
> --mount "type=volume,src=my_first_volume,target=/home/app" \
> jedha/volumes_demo sh
> ```

run `ls`, you should still see `hello.py` eventhough you stopped your container! 

However, it didn't create a new `hello.py` on your host machine at the current directory you are in. This is because the file you created is managed by Docker and stored somewhere else! The only way to use the file is by binding it to a container. 

Now, if you want to remove `volumes` simply run:

`docker volume rm VOLUME_NAME`

## Volumes or Bind mounts?

To summarize your decision process: 

![snap](https://full-stack-assets.s3.eu-west-3.amazonaws.com/Deployment/volumes_or_bind_mounts.svg)

If you want to use code and be able to modify it because you are in a development environment, you should use `bind mounts`. In any other cases, you should use normal `volumes`.

## Resources 📚📚

* [Storage types](https://docs.docker.com/storage/)
* [Use Volumes](https://docs.docker.com/storage/volumes/)